In [1]:
from tensorflow.keras.layers import Input, Conv1D, Dense, Dropout, Lambda, concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.layers import Dense, Dropout, LSTM, GRU, BatchNormalization, Input, RepeatVector, TimeDistributed, Conv1D, MaxPooling1D, Flatten, AveragePooling1D, Activation, GlobalAveragePooling1D
from tensorflow.keras.models import Sequential, Model
import tensorflow as tf
import random
import numpy as np
from collections import deque
from math import sqrt
import modin.pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-04-01_12-25-49_2778/logs.
Waiting for redis server at 127.0.0.1:54345 to respond...
Waiting for redis server at 127.0.0.1:10742 to respond...
Starting Redis shard with 10.0 GB max memory.
Starting the Plasma object store with 4.0 GB memory using /dev/shm.


In [18]:
df = pd.read_csv('daily-minimum-temperatures-in-me.csv', error_bad_lines=False, header=None, index_col=0)
#names = pd.read_csv("regression/Twitter/Twitter.names", header=None)

In [19]:
df.head()
#df=df.astype('float32')
df.info()

<class 'modin.pandas.dataframe.DataFrame'>
Index: 3652 entries, Date to Daily minimum temperatures in Melbourne
Data columns (total 1 columns):
1    3652 non-null object
dtypes: object(1)
memory usage: 57.1+ KB


In [20]:
df = df.iloc[:, 77]

IndexError: positional indexers are out-of-bounds

In [21]:
# transform list into supervised learning format
def series_to_supervised(data, n_in=1, n_out=1):
    cols = list()
    #input sequence (t-n, ..., t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
    #forecast sequence
    for i in range(0, n_out):
        cols.append(df.shift(-i))
    #put it all together
    agg = pd.concat(cols, axis=1)
    #drop rows with nan values
    agg.dropna(inplace=True)
    return agg

In [6]:
df = series_to_supervised(df,12,1)

In [7]:
df.head(14)

,77,77,77,77,77,77,77,77,77,77,77,77,77
12,0.0,0.5,0.0,2.5,0.5,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5
13,0.5,0.0,2.5,0.5,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5
14,0.0,2.5,0.5,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5
15,2.5,0.5,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5
16,0.5,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5
17,1.0,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5,12.0
18,0.0,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5,12.0,8.0
19,2.0,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5,12.0,8.0,1.0
20,1.0,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5,12.0,8.0,1.0,6.0
21,0.5,0.5,0.0,17.5,0.5,6.5,28.5,23.5,12.0,8.0,1.0,6.0,1.0


In [37]:
def forward_evaluate_plot(df, lookback):
    scaler = MinMaxScaler()
    train, test = train_test_split(df.values)
    X, y = train[:,:-1], train[:, -1]

    X = scaler.fit_transform(X)
    X = X.reshape(X.shape[0], X.shape[1], 1)
    model, history = fit_model(X, y)
    
    X_test = scaler.fit_transform(test[:,:-1])
    
    predictions = list()
    actual = list()
    for i in range(len(test)):

        x_input = X_test[i,:]
        x_input = x_input.reshape((1, lookback, 1))
        yhat = model.predict(x_input)
        predictions.append(yhat[0])
        actual.append(test[i,-1:])

    rmse, mape = evaluate(predictions, actual)
    


    
    plt.figure(figsize=(16,10))
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])

    plt.xlabel('Epoch')
    plt.ylabel('Mean Squared Error Loss')
    plt.title('Loss Over Time')
    plt.legend(['Train','Valid'])

    plt.figure(figsize=(13,10))
    plt.plot(predictions[-100:])
    plt.plot(actual[-100:])
    
    plt.xlabel('Time')
    plt.ylabel('DataRate (MB)')
    plt.title('DataRate Over Time')
    plt.legend(['Prediction','Actual'])
    
    print(f"RMSE = {rmse}")
    print(f"MAE = {mape}") 

In [38]:
def fit_model(train_x, train_y):

    NAME = 'CNN_optimized'

    inputs = Input(shape=(train_x.shape[1:]))

    x = Conv1D(128, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256, 2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128, 1, padding='same')(x)
    #x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    
    x = GlobalAveragePooling1D()(x)

    prediction = Dense(1)(x)

    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

    model = Model(inputs, prediction)

    model.compile(loss='mae',
                  optimizer=opt)

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=50,
                        validation_split=0.1,
                        verbose=1)

    return model, history

In [39]:
def evaluate(predictions, actual):
    mse = mean_squared_error(actual,predictions)
    mae = mean_absolute_error(actual, predictions)
    rmse = sqrt(mse)
    return rmse, mae

In [40]:
forward_evaluate_plot(df, 12)

Train on 393685 samples, validate on 43743 samples
Epoch 1/50
393685/393685 [==============================] - 162s 412us/step - loss: 55.9988 - val_loss: 2297.8572
Epoch 2/50
393685/393685 [==============================] - 162s 410us/step - loss: 48.3947 - val_loss: 2534.4962
Epoch 3/50
281056/393685 [====================>.........] - ETA: 45s - loss: 47.4488

KeyboardInterrupt: 

In [ ]:
def fit_model(train_x, train_y):

    NAME = 'GRU'

    inputs = Input(shape=(train_x.shape[1:]))

    x = GRU(64, activation='relu', return_sequences=True)(inputs)
    
    x = GRU(64, activation='relu')(x)

    prediction = Dense(1)(x)

    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

    model = Model(inputs, prediction)

    model.compile(loss='mae',
                  optimizer=opt)

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=100,
                        validation_split=0.1,
                        verbose=1)

    return model, history

In [ ]:
forward_evaluate_plot(df, 12)

In [ ]:
def fit_model(train_x, train_y):

    NAME = 'ResNet'

    inputs = Input(shape=(train_x.shape[1:]))

    x = Conv1D(128, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(256, 2, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    x = Conv1D(128, 1, padding='same')(x)
    #x = BatchNormalization()(x)
    #x = Activation('relu')(x)
    x = concatenate([x, inputs])
    
    y = Conv1D(128, 3, padding='same')(inputs)
    y = BatchNormalization()(x)
    y = Activation('relu')(x)

    y = Conv1D(256, 2, padding='same')(x)
    y = BatchNormalization()(x)
    y = Activation('relu')(x)

    y = Conv1D(128, 1, padding='same')(x)
    
    y= concatenate([y, x])
    
    y = GlobalAveragePooling1D()(y)

    prediction = Dense(1)(y)

    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

    model = Model(inputs, prediction)

    model.compile(loss='mae',
                  optimizer=opt)

    history = model.fit(train_x, train_y,
                        batch_size=32,
                        epochs=50,
                        validation_split=0.1,
                        verbose=1)

    return model, history

In [ ]:
forward_evaluate_plot(df, 12)